# Método de Monte Carlo

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as py
import finoptions as fo
import warnings
warnings.filterwarnings('ignore')

## Opções Européias

In [4]:
# Simulação Monte Carlo - Opções Europeias tipo Call
S0 = 50
K = 49
sigma = 0.3
T=3/12
N=10 #N=1
tt = T/N
r = 0.06
M=1000000
z =  np.random.normal(loc=0, scale=1, size=(N, M))
S = S0 * np.exp(np.cumsum((r-0.5*sigma**2)*tt
+ sigma * np.sqrt(tt) * z, axis=0))
S_T = S[-1]
V_T = np.maximum(S_T-K,0)
V_price_MC = np.mean(np.exp(-r*T)*V_T)
# Compara com B&S
opt1_BS = fo.BlackScholesOption(S0,K,T,r,r,sigma)
V_price_BS = opt1_BS.call()
print("MMC: ",V_price_MC)
print("B&S: ",V_price_BS)

MMC:  3.8862620201843336
B&S:  3.8826838282197933


## Opções Americanas



In [5]:
# Simulação Monte Carlo - Opções Americanas tipo PUT (Método Longstaff & Schwartz)
S0 = 50
K = 52
sigma = 0.3
r = 0.05
M=100000
T=2
N =200
tt = T / N
disc = np.exp(-r * tt)
# Simulações das trajetórias de preço do ativo objeto
z =  np.random.normal(loc=0, scale=1, size=(N, M))
S = S0 * np.exp(np.cumsum((r-0.5*sigma**2)*tt
+ sigma * np.sqrt(tt) * z, axis=0))
S[0] = S0
# PUT Americana
Exer = np.maximum(K - S, 0)
# Valor no vencimento
V = Exer[-1] #V_T, mas será utilizada recursivamente
for t in range(N - 1, 0, -1):
    idx = Exer[t]>0
    coefs = np.polyfit(S[t,idx], V[idx]*disc, 5)
    f = np.polyval(coefs, S[t,idx])
    V[idx] = np.where(Exer[t,idx] > f, Exer[t,idx], V[idx]*disc)
V_price_LSM = disc * np.mean(V)
# Compara com CRR
opt1_CRR = fo.binomial_tree_options.CRRBinomialTreeOption(S0,K,T,r,r,sigma,'american',N)
V_price_CRR = opt1_CRR.put()
opt2_BAW = fo.basic_american_options.BAWAmericanApproxOption(S0, K, T, r, r, sigma)
V_price_BAW = opt2_BAW.put()
print("LSM: ",V_price_LSM)
print("CRR: ",V_price_CRR)
print("BAW: ",V_price_BAW)

LSM:  7.575188263694922
CRR:  7.477208328936235
BAW:  7.496568454380279
